
<br>
=================================================<br>
Non-local means denoising for preserving textures<br>
=================================================<br>
In this example, we denoise a detail of the astronaut image using the non-local<br>
means filter. The non-local means algorithm replaces the value of a pixel by an<br>
average of a selection of other pixels values: small patches centered on the<br>
other pixels are compared to the patch centered on the pixel of interest, and<br>
the average is performed only for pixels that have patches close to the current<br>
patch. As a result, this algorithm can restore well textures, that would be<br>
blurred by other denoising algorithm.<br>
When the ``fast_mode`` argument is ``False``, a spatial Gaussian weighting is<br>
applied to the patches when computing patch distances.  When ``fast_mode`` is<br>
``True`` a faster algorithm employing uniform spatial weighting on the patches<br>
is applied.<br>
For either of these cases, if the noise standard deviation, ``sigma``, is<br>
provided, the expected noise variance is subtracted out when computing patch<br>
distances.  This can lead to a modest improvement in image quality.<br>
The ``estimate_sigma`` function can provide a good starting point for setting<br>
the ``h`` (and optionally, ``sigma``) parameters for the non-local means algorithm.<br>
``h`` is a constant that controls the decay in patch weights as a function of the<br>
distance between patches.  Larger ``h`` allows more smoothing between disimilar<br>
patches.<br>
In this demo, ``h``, was hand-tuned to give the approximate best-case performance<br>
of each variant.<br>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise

In [ ]:
astro = img_as_float(data.astronaut())
astro = astro[30:180, 150:300]

In [ ]:
sigma = 0.08
noisy = random_noise(astro, var=sigma**2)

estimate the noise standard deviation from the noisy image

In [ ]:
sigma_est = np.mean(estimate_sigma(noisy, channel_axis=-1))
print(f"estimated noise standard deviation = {sigma_est}")

In [ ]:
patch_kw = dict(patch_size=5,      # 5x5 patches
                patch_distance=6,  # 13x13 search area
                channel_axis=-1)

slow algorithm

In [ ]:
denoise = denoise_nl_means(noisy, h=1.15 * sigma_est, fast_mode=False,
                           **patch_kw)

slow algorithm, sigma provided

In [ ]:
denoise2 = denoise_nl_means(noisy, h=0.8 * sigma_est, sigma=sigma_est,
                            fast_mode=False, **patch_kw)

fast algorithm

In [ ]:
denoise_fast = denoise_nl_means(noisy, h=0.8 * sigma_est, fast_mode=True,
                                **patch_kw)

fast algorithm, sigma provided

In [ ]:
denoise2_fast = denoise_nl_means(noisy, h=0.6 * sigma_est, sigma=sigma_est,
                                 fast_mode=True, **patch_kw)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 6),
                       sharex=True, sharey=True)

In [ ]:
ax[0, 0].imshow(noisy)
ax[0, 0].axis('off')
ax[0, 0].set_title('noisy')
ax[0, 1].imshow(denoise)
ax[0, 1].axis('off')
ax[0, 1].set_title('non-local means\n(slow)')
ax[0, 2].imshow(denoise2)
ax[0, 2].axis('off')
ax[0, 2].set_title('non-local means\n(slow, using $\\sigma_{est}$)')
ax[1, 0].imshow(astro)
ax[1, 0].axis('off')
ax[1, 0].set_title('original\n(noise free)')
ax[1, 1].imshow(denoise_fast)
ax[1, 1].axis('off')
ax[1, 1].set_title('non-local means\n(fast)')
ax[1, 2].imshow(denoise2_fast)
ax[1, 2].axis('off')
ax[1, 2].set_title('non-local means\n(fast, using $\\sigma_{est}$)')

In [ ]:
fig.tight_layout()

print PSNR metric for each case

In [ ]:
psnr_noisy = peak_signal_noise_ratio(astro, noisy)
psnr = peak_signal_noise_ratio(astro, denoise)
psnr2 = peak_signal_noise_ratio(astro, denoise2)
psnr_fast = peak_signal_noise_ratio(astro, denoise_fast)
psnr2_fast = peak_signal_noise_ratio(astro, denoise2_fast)

In [ ]:
print(f"PSNR (noisy) = {psnr_noisy:0.2f}")
print(f"PSNR (slow) = {psnr:0.2f}")
print(f"PSNR (slow, using sigma) = {psnr2:0.2f}")
print(f"PSNR (fast) = {psnr_fast:0.2f}")
print(f"PSNR (fast, using sigma) = {psnr2_fast:0.2f}")

In [ ]:
plt.show()